In [1]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'Demos/CarPrice_Assignment.csv'
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
car_price = pd.read_csv(file_content_stream)
car_price.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [20]:
## Defining the input and target variables
X = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 
               'peakrpm', 'citympg', 'highwaympg']]
Y = car_price['price']

## Defining the list to store estimated coefficients at each split
coef = list()

for i in range(0, 1000):
    
    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    ## Running LASSO cross-validation to estimate optimal lambda
    lasso_cv = LassoCV(normalize = True, cv = 5).fit(X_train, Y_train)

    ## Building LASSO regression with optimal lambda
    lasso_md = Lasso(alpha = lasso_cv.alpha_, 
                     normalize = True).fit(X_train, Y_train)

    ## Storing estimated coefficients
    coef.append(lasso_md.coef_)

## Putting the list as data-frame
coef_data = pd.DataFrame(coef)

In [21]:
coef_data.head()

,0,1,2,3,4,5,6
0,102.331078,127.240598,337.952790,40.191600,2.102311,-144.668692,0.000000
1,150.637605,107.730350,268.446368,46.497583,1.966640,-132.028617,-0.000000
2,118.018906,108.724370,252.353526,57.339576,1.407824,-34.257229,-43.609474
3,173.931895,115.632914,324.610461,42.606939,2.236615,-61.671234,-48.106707
4,149.017949,125.572121,275.214416,40.484087,2.259357,-84.960466,0.000000


In [29]:
sum(coef_data[6] == 0.0)

758

In [30]:
## Defining the input and target variables
X = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 
               'peakrpm', 'citympg']]
Y = car_price['price']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)